In [1]:
import os
import time
import pandas as pd
import numpy as np
import networkx as nx
import collections
from scipy import sparse as sp
from scipy.stats import rankdata

import itertools
from itertools import combinations, combinations_with_replacement, cycle
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from util import *

import colorcet as cc

import bokeh
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.models import (Rect, MultiLine, Circle, Span, Label,
                          GraphRenderer, StaticLayoutProvider,
                          NodesAndLinkedEdges,
                          HoverTool, TapTool, ColumnDataSource,
                          LinearColorMapper, LogColorMapper, CategoricalColorMapper,
                          CategoricalMarkerMapper,
                          BoxSelectTool,
                          ColorBar, BasicTicker, BoxZoomTool, FactorRange,
                          Range1d)
from bokeh.models import CategoricalTicker, FixedTicker, BoxAnnotation
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead, LabelSet

from bokeh.transform import transform, factor_cmap, linear_cmap, log_cmap
from bokeh.layouts import row, column, gridplot
output_notebook()

Loading BokehJS ...

In [22]:
def joint_marginal(df, c1, c2, include_fraction=False):
    """Given a dataframe and two columns, return a dataframe with the joint and marginal counts."""
    j = df.value_counts([c1, c2])
    j.name = "joint_count"
    j = j.reset_index()

    m1 = df.value_counts(c1)
    m1.name = f"{c1}_count"
    j = j.merge(m1, left_on=c1, right_index=True)

    m2 = df.value_counts(c2)
    m2.name = f"{c2}_count"
    j = j.merge(m2, left_on=c2, right_index=True)

    if include_fraction:
        j["joint_fraction"] = j["joint_count"] / j["joint_count"].sum()
        j[f"{c1}_fraction"] = j["joint_count"] / j[f"{c1}_count"]
        j[f"{c2}_fraction"] = j["joint_count"] / j[f"{c2}_count"]
    return j


hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "APL/preprocessed-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "APL/clustering_" + hemibrain_version
hemibrain_nodes = "apl_full_key.txt"
hemibrain_edges = "apl_full.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.1','0.25','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

2023 10 20 14:13:35  Hemibrain data set being used: v1.2.1
2023 10 20 14:13:35  Set up directory info and useful lists


In [23]:
from neuprint import Client
from neuprint import fetch_roi_hierarchy, fetch_neurons, NeuronCriteria as NC


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")
pd.set_option('display.max_rows', 200)

2023 10 20 14:13:37  neuprint Client set up as `np_Client`, version 0.1.0
2023 10 20 14:13:37  Loading node dataframe
2023 10 20 14:13:37    (without centroids)
2023 10 20 14:13:37  Adding 'type group'
2023 10 20 14:13:37  Done!
2023 10 20 14:13:37  Loading directed edges from csv
2023 10 20 14:13:38  Done!
2023 10 20 14:13:38  Merging in cell info to edge df
2023 10 20 14:13:38  Done!


In [24]:
chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   #display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

g = gridplot(fs, ncols=2)
show(g)

Clusters found at chi = 0.0: 4
Clusters found at chi = 0.1: 86
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.25: 195
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.5: 271
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.75: 376


In [26]:
output_file("/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_full.html")
save(g, title='Joint Marginal Analysis (Full)')

'/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_full.html'

In [16]:
# Inputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "APL/preprocessed_inputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "APL/clustering_inputs_" + hemibrain_version
hemibrain_nodes = "apl_in_key.txt"
hemibrain_edges = "apl_in.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.1','0.25','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   #display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

k = gridplot(fs, ncols=2)
show(k)

2023 10 20 14:09:42  Hemibrain data set being used: v1.2.1
2023 10 20 14:09:42  Set up directory info and useful lists
2023 10 20 14:09:42  neuprint Client set up as `np_Client`, version 0.1.0
2023 10 20 14:09:42  Loading node dataframe
2023 10 20 14:09:42    (without centroids)
2023 10 20 14:09:42  Adding 'type group'
2023 10 20 14:09:42  Done!
2023 10 20 14:09:42  Loading directed edges from csv
2023 10 20 14:09:43  Done!
2023 10 20 14:09:43  Merging in cell info to edge df
2023 10 20 14:09:43  Done!
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.1: 66
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.25: 128
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.5: 200
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.75: 249


In [17]:
# outputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "APL/preprocessed_outputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "APL/clustering_outputs_" + hemibrain_version
hemibrain_nodes = "apl_out_key.txt"
hemibrain_edges = "apl_out.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.1','0.25','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   #display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

i = gridplot(fs, ncols=2)
show(i)

2023 10 20 14:11:08  Hemibrain data set being used: v1.2.1
2023 10 20 14:11:08  Set up directory info and useful lists
2023 10 20 14:11:09  neuprint Client set up as `np_Client`, version 0.1.0
2023 10 20 14:11:09  Loading node dataframe
2023 10 20 14:11:09    (without centroids)
2023 10 20 14:11:09  Adding 'type group'
2023 10 20 14:11:09  Done!
2023 10 20 14:11:09  Loading directed edges from csv
2023 10 20 14:11:09  Done!
2023 10 20 14:11:09  Merging in cell info to edge df
2023 10 20 14:11:10  Done!
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.1: 67
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.25: 148
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.5: 200
Clusters found at chi = 0.0: 4
Clusters found at chi = 0.75: 236


In [19]:
output_file("/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_outputs.html")
save(i, title='Joint Marginal Analysis (Ouputs)')

'/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_outputs.html'

In [21]:
output_file("/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_inputs.html")
save(k, title='Joint Marginal Analysis (Inputs)')

'/Users/rhessa/flybrain-clustering/APL/figures/joint_marginal_comparison_inputs.html'